<a href="https://colab.research.google.com/github/omuratgultekin/Welcome/blob/main/lasversion12_mod2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
from typing import Dict, List, Tuple, TypedDict
from dataclasses import dataclass
from enum import Enum

class TransportMode(Enum):
    ROAD = "road"
    SEA = "sea"
    RAIL = "rail"
    ROAD_AND_SEA = "road+sea"

class Direction(Enum):
    FORWARD = "forward"
    BACKWARD = "backward"
    RIGHT = "right"
    LEFT = "left"

class Halat(TypedDict):
    Halat_Türü: str
    TOKA_Türü: str
    D_Lashing_MSL: float

@dataclass
class LashingResult:
    halat_type: str
    buckle_type: str
    lashing_msl: float
    num_lashings: int

@dataclass
class DirectionalResult:
    lashing: LashingResult | None
    final_force: float
    reason: str | None = None

class LoadSecuringCalculator:
    def __init__(
        self,
        initial_forces: Dict[str, float],
        available_halats: List[Halat],
        transport_mode: TransportMode,
        loop_lash: bool = False
    ):
        self.initial_forces = initial_forces
        self.available_halats = available_halats
        self.transport_mode = transport_mode
        self.loop_lash = loop_lash
        self.direction_order = self._get_direction_order()
        self.results: Dict[Direction, DirectionalResult] = {}

    def _get_direction_order(self) -> List[Direction]:
        """Determines the order of securing directions based on transport mode."""
        direction_orders = {
            TransportMode.ROAD: [Direction.FORWARD, Direction.BACKWARD, Direction.RIGHT, Direction.LEFT],
            TransportMode.SEA: [Direction.RIGHT, Direction.LEFT, Direction.FORWARD, Direction.BACKWARD],
            TransportMode.RAIL: [Direction.FORWARD, Direction.BACKWARD, Direction.RIGHT, Direction.LEFT],
            TransportMode.ROAD_AND_SEA: [Direction.FORWARD, Direction.RIGHT, Direction.LEFT, Direction.BACKWARD]
        }
        return direction_orders[self.transport_mode]

    def _calculate_cross_force(self, direction: Direction) -> float:
        """Calculates cross forces from perpendicular lashings for a given direction."""
        cross_force = 0

        if direction in [Direction.RIGHT, Direction.LEFT]:
            # Forward ve Backward halatların sağ/sol yöndeki etkisi
            for source_dir in [Direction.FORWARD, Direction.BACKWARD]:
                if source_dir in self.results and self.results[source_dir].lashing:
                    cross_force += (
                        self.results[source_dir].lashing.num_lashings *
                        self.results[source_dir].lashing.lashing_msl / 2
                    )

        elif direction in [Direction.FORWARD, Direction.BACKWARD]:
            # Sağ ve sol halatların forward/backward yöndeki etkisi
            for source_dir in [Direction.RIGHT, Direction.LEFT]:
                if source_dir in self.results and self.results[source_dir].lashing:
                    cross_force += (
                        self.results[source_dir].lashing.num_lashings *
                        self.results[source_dir].lashing.lashing_msl / 2
                    )

        return cross_force

    def _find_optimal_halat(self, required_force: float) -> Tuple[LashingResult | None, int]:
        """Finds the optimal halat and number of lashings needed for a given force."""
        if required_force <= 0:
            return None, 0

        optimal_result = None
        min_lashings = float('inf')

        for halat in self.available_halats:
            lashing_msl = halat["D_Lashing_MSL"]
            lashings_needed = math.ceil(required_force / lashing_msl)

            # Ensure even number of lashings with minimum of 2
            lashings_needed = max(2, (lashings_needed + 1) & ~1)

            if lashings_needed < min_lashings:
                min_lashings = lashings_needed
                optimal_result = LashingResult(
                    halat_type=halat["Halat_Türü"],
                    buckle_type=halat["TOKA_Türü"],
                    lashing_msl=lashing_msl,
                    num_lashings=lashings_needed
                )

        return optimal_result, min_lashings

    def calculate_optimal_lashings(self) -> Dict[Direction, DirectionalResult]:
        """Calculates optimal lashing configuration for all directions."""
        for direction in self.direction_order:
            initial_force = self.initial_forces[f"net_initial_{direction.value}_force"]
            cross_force = self._calculate_cross_force(direction)

            # Net initial force'u cross force'a göre ayarla
            adjusted_force = max(0, initial_force - cross_force)

            optimal_lashing, num_lashings = self._find_optimal_halat(adjusted_force)

            reason = None
            if not optimal_lashing:
                reason = (
                    "Initial force is zero."
                    if initial_force == 0
                    else "Remaining force is already satisfied by other lashings."
                )

            self.results[direction] = DirectionalResult(
                lashing=optimal_lashing,
                final_force=0,  # Geçici değer, final_forces'da güncellenecek
                reason=reason
            )

        # Ensure minimum backward lashings for road and rail transport
        if (self.transport_mode in (TransportMode.ROAD, TransportMode.RAIL) and
            not self.results[Direction.BACKWARD].lashing):
            smallest_halat = min(self.available_halats, key=lambda x: x["D_Lashing_MSL"])
            self.results[Direction.BACKWARD] = DirectionalResult(
                lashing=LashingResult(
                    halat_type=smallest_halat["Halat_Türü"],
                    buckle_type=smallest_halat["TOKA_Türü"],
                    lashing_msl=smallest_halat["D_Lashing_MSL"],
                    num_lashings=2
                ),
                final_force=0  # Geçici değer, final_forces'da güncellenecek
            )

        # Final force'ları hesapla
        self._calculate_final_forces()

        return self.results

    def _calculate_final_forces(self):
        """Calculates final forces for all directions after all lashings are determined."""
        for direction in self.direction_order:
            initial_force = self.initial_forces[f"net_initial_{direction.value}_force"]

            # Mevcut yöndeki halatların etkisi
            direct_force = 0
            if self.results[direction].lashing:
                direct_force = (
                    self.results[direction].lashing.num_lashings *
                    self.results[direction].lashing.lashing_msl
                )

            # Cross force'ları hesapla
            cross_force = self._calculate_cross_force(direction)

            # Final force = Initial force - Direct force - Cross force
            final_force = initial_force - direct_force - cross_force

            # Sonucu güncelle
            self.results[direction] = DirectionalResult(
                lashing=self.results[direction].lashing,
                final_force=final_force,
                reason=self.results[direction].reason
            )

    def format_results(self) -> str:
        """Formats the calculation results into a human-readable string."""
        output = " ---Optimal Lashing Plan----\n\n"

        for direction in self.direction_order:
            result = self.results[direction]
            output += f"----{direction.value.capitalize()} Direction----\n"

            if result.lashing:
                output += (
                    f"Halat Türü  : {result.lashing.halat_type}"
                    f" / {result.lashing.buckle_type}\n"
                    f"Lashing MSL : {result.lashing.lashing_msl}"
                    f" / Number of Lashings: {result.lashing.num_lashings}\n"
                    f"Final Net Force: {result.final_force:.2f}\n\n"
                )
            else:
                output += (
                    f"Status: No lashing required.\n"
                    f"Reason: {result.reason}\n"
                    f"Final Net Force: {result.final_force:.2f}\n\n"
                )

        return output


In [3]:
initial_forces = {
    "net_initial_forward_force": 40,
    "net_initial_backward_force":0,
    "net_initial_right_force": 0,
    "net_initial_left_force": 0,
}

available_halats = [
    {"Halat_Türü": "CL105", "TOKA_Türü": "HDB1ON", "D_Lashing_MSL": 10},
    {"Halat_Türü": "CL150", "TOKA_Türü": "HDB12C", "D_Lashing_MSL": 15},
    {"Halat_Türü": "CL200", "TOKA_Türü": "HDB12C", "D_Lashing_MSL": 20},
    {"Halat_Türü": "CL200", "TOKA_Türü": "HDB12", "D_Lashing_MSL": 21.25},
    {"Halat_Türü": "CL750", "TOKA_Türü": "HDB15N", "D_Lashing_MSL": 50},
    {"Halat_Türü": "CL1500", "TOKA_Türü": "DYNA20", "D_Lashing_MSL": 100},
]

calculator = LoadSecuringCalculator(
    initial_forces=initial_forces,
    available_halats=available_halats,
    transport_mode=TransportMode.ROAD,
    loop_lash=False
)

results = calculator.calculate_optimal_lashings()
formatted_output = calculator.format_results()
print(formatted_output)

 ---Optimal Lashing Plan----

----Forward Direction----
Halat Türü  : CL200 / HDB12C
Lashing MSL : 20 / Number of Lashings: 2
Final Net Force: 0.00

----Backward Direction----
Halat Türü  : CL105 / HDB1ON
Lashing MSL : 10 / Number of Lashings: 2
Final Net Force: -20.00

----Right Direction----
Status: No lashing required.
Reason: Initial force is zero.
Final Net Force: -30.00

----Left Direction----
Status: No lashing required.
Reason: Initial force is zero.
Final Net Force: -30.00


